# Object Detection
This notebook will serve as the primary means of documentation for creating an object detection model
_______________________________________________________________________________________________________________________________________________

### Full group includes:
- Jordan Brown
- Dylan Roy
- Maxwell Barret
- Julia Dewhurt

### Primary notebook contributers:
- ""
- ""
_______________________________________________________________________________________________________________________________________________

***The primary notebook contributers are the group members who were assigned to this specific task. All group members will work in collaboration to create a final working product. However, the nature of this project calls for the full group to be assigned to primary tasks.***

## Notebbook Goal/Purpose

This notebook will be used to create a model for traffic based object detection. This will involve using labelled image and video feeds to determine what traffic objects look like. This model will hoepefully be able to detect and track traffic counts at individual intersections. This tracking will be used to create a dataset which can hopefully be used to train a traffic prediction for the traffic analysis portion of this project.
_______________________________________________________________________________________________________________________________________________

Code sells should use the following layout template
```
"""
At the top will be a commment block
this comment block will describe the
primary purpose for the cells code.
"""

# A comment stating used libary(s)

Code
    Implementation
        Goes
            Here

```

In [5]:
"""
This cell will contain the import statements for the notebook.
"""
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import json
import pandas as pd

In [6]:
"""
This cell will import the datasets that will be used in this project.
"""

# from kaggle.api.kaggle_api_extended import KaggleApi
# import os

# Initialize the API
api = KaggleApi()
api.authenticate()

# Dataset details
dataset = "imtkaggleteam/city-intersection-computer-vision"
download_path = "dataset"  # Desired directory

# Ensure the download path exists
if not os.path.exists(download_path):
    os.makedirs(download_path, exist_ok=True)

    # Download the dataset
    api.dataset_download_files(dataset, path=download_path, unzip=True)

    print(f"Dataset downloaded to: {os.path.abspath(download_path)}")


In [7]:
# a function that used for creation a coco json file from a csv file
# original reference -> https://stackoverflow.com/questions/62545034/how-to-convert-a-csv-table-into-coco-format-in-python

def create_csv_to_coco(path, save_json_path):
    data = pd.read_csv(path)
    data = data[['filename','class','width', 'height','xmin','ymin','xmax','ymax']]

    images = []
    categories = []
    annotations = []

    category = {}
    category["supercategory"] = 'none'
    category["id"] = 0
    category["name"] = 'None'
    categories.append(category)

    data['fileid'] = data['filename'].astype('category').cat.codes
    data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
    data['categoryid'] = data['categoryid']+1
    data['annid'] = data.index
    
    def image(row):
        image = {}
        image["height"] = row.height
        image["width"] = row.width
        image["id"] = row.fileid
        image["file_name"] = row.filename
        return image

    def category(row):
        category = {}
        category["supercategory"] = 'None'
        category["id"] = row.categoryid
        category["name"] = row[2]
        return category

    def annotation(row):
        annotation = {}
        area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
        annotation["segmentation"] = []
        annotation["iscrowd"] = 0
        annotation["area"] = area
        annotation["image_id"] = row.fileid

        annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

        annotation["category_id"] = row.categoryid
        annotation["id"] = row.annid
        return annotation

    for row in data.itertuples():
        annotations.append(annotation(row))

    imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
    for row in imagedf.itertuples():
        images.append(image(row))

    catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
    for row in catdf.itertuples():
        categories.append(category(row))

    data_coco = {}
    data_coco["images"] = images
    data_coco["categories"] = categories
    data_coco["annotations"] = annotations
    json.dump(data_coco, open(save_json_path, "w"), indent=4)
    print( save_json_path ,' file created...')

In [ ]:
create_csv_to_coco(f'{download_path}/train/_annotations.csv', 
                   'dataset/train_coco.json')
create_csv_to_coco(f'{download_path}/valid/_annotations.csv', 
                   'dataset/valid_coco.json')
create_csv_to_coco(f'{download_path}/test/_annotations.csv', 
                   'dataset/test_coco.json')

FileNotFoundError: [Errno 2] No such file or directory: 'test/train_coco.json'